<a href="https://colab.research.google.com/github/migperfer/MIR-UPF/blob/master/best_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and import the python packages

In [0]:
!pip install  essentia numpy matplotlib
import os
import numpy as np
import matplotlib as plt
import essentia.standard as std
import pandas as pd
if not os.path.isdir('TIVlib'):
    !git clone https://github.com/aframires/TIVlib
else:
    print("TIVlib already installed")
from TIVlib import TIVlib
from glob import glob
from IPython.display import display, Audio, HTML
import re
import csv
from zipfile import ZipFile
import requests

## Download and extract the folder contaning the audio loops

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
file_id = '1mS0G_Gk4v6IHe2E2IZ2Z6dcE-lcTuZx3'
destination = 'audios.zip'
download_file_from_google_drive(file_id, destination)

In [0]:
# Unzip the file
with ZipFile('audios.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall()

## Get all the mp3 files and select a target audio

#### Retrieve all possible loops

In [0]:
mp3list = glob('subset/*.mp3')

## Define a function to get beatwise TIVs

In [0]:
def get_beat_chunks(filename, bpm_restrict=None):
    audio = std.MonoLoader(filename=filename)()
    hpcp = std.HPCP()
    spectrum = std.Spectrum()
    speaks = std.SpectralPeaks()
    tivs = []
    sr = 44100
    bpm = get_tempo(filename)
    if bpm_restrict != None and bpm_restrict!=bpm:
        raise ValueError
    sec_beat = (60/bpm)
    beats = np.arange(0, len(audio)/sr, sec_beat)
    beats = np.append(beats, len(audio)/sr)
    for i in range(1, len(beats)):
        segmented_audio = audio[int(beats[i - 1] * sr):int(beats[i] * sr)]
        cutter = std.FrameGenerator(segmented_audio)
        aux = []
        for spec in cutter:
            freq, mag = speaks(spec)
            chroma = hpcp(freq, mag)
            aux.append(chroma)
        chroma = np.mean(aux, axis=1)
        tiv = TIVlib.TIV.from_pcp(chroma)
        tivs.append(tiv)
    return tivs

def get_number_beats(filename):
    audio = std.MonoLoader(filename=filename)()
    sr = 44100
    bpm = get_tempo(filename)
    sec_beat = (60/bpm)
    beats = np.arange(0, len(audio)/sr, sec_beat)
    beats = np.append(beats, len(audio)/sr)
    return len(beats)

def get_tempo(filename):
    try:
        bpm = int(re.search(r"(\d+)bpm", filename).group(1))
    except:
        bpm = int(re.search(r"/(\d+)-", filename).group(1))
    return bpm

## Get an overview of the dataset

In [0]:
df = []
if not os.path.isfile('compt_loops.csv'):
  for song in mp3list:
    df.append({'filename': song, 'n_beats': get_number_beats(song), 'tempo': get_tempo(song)})
  df = pd.DataFrame(df)
  df.head()
  idx = df.groupby(['n_beats','tempo']).count().idxmax()
  n_beats_max = idx[0][0]
  tempo_max = idx[0][1]
  final_df = (df['n_beats'] == n_beats_max) & (df['tempo'] == tempo_max)
  final_df = df[final_df]
  print("Maximum number of common feature samples for %s number of beats and %s bpm. A total of %s loops" % 
        (n_beats_max, tempo_max, len(final_df)))
  mp3list = final_df['filename'].tolist()
  final_df.to_csv('compt_loops.csv')
  final_df.head()

### Select a target audio randomly
Listen to the target loop. If it's mainly a drum loop, rerun this cell until you get something that doesn't contain mainly percussion.



In [0]:
# Target audio selected using the initial form
loop_sample = np.random.randint(0, 918)
target_audio = mp3list[loop_sample]
display(Audio(filename=target_audio))

## Load all the compatible loops

In [0]:
filenames = []
with open('compt_loops.csv', 'r') as file:
    dicw = csv.DictReader(file)
    for row in dicw:
        filenames.append(row['filename'])

In [0]:
compdict = {}  # A dictionary to keep al compatibilities
for candidate in filenames:
    tivcand = get_beat_chunks(candidate)
    comp = []
    for i in range(len(main_song_tivs)):
        comp.append(main_song_tivs[i].small_scale_compatibility(tivcand[i]))
        compdict[candidate] = np.sum(comp)

### Sort the loops according to compatibility
Create also a dictionary containing the 10 most compatibles

In [0]:
compdict = {k: v for k, v in sorted(compdict.items(), key=lambda item: item[1])}
dict_10 = {}
for x in list(compdict)[1:11]:
    dict_10[x] = compdict[x]

In [0]:
def create_mix(song1, song2):
    audio1 = std.MonoLoader(filename=song1)()
    audio2 = std.MonoLoader(filename=song2)()
    try:
      mix = audio1 + audio2[:len(audio1)]
    except:
      mix = audio2 + audio1[:len(audio2)]
    return mix

## Create the mixes and listen to them

In [0]:
i = 0
for key in dict_10.keys():
    display(HTML("<h3>Mix #%s: %s</h3>" % (i, key.split('/')[-1])))
    display(HTML("Original"))
    display(Audio(filename=key))
    mix = create_mix(target_audio, key)
    display(HTML("Mix"))
    display(Audio(data=mix, rate=44100))
    i += 1

# Results

In [0]:
#@title Rate the _consonance_ of the mixes with a score between 0 and 5
#@markdown Run this cell to see the output.
#@markdown Please rate the mix with a **-1** if the candidate loop for the mix is mainly a drum loop
mix_0 = 2  #@param {type: "slider", min: -1, max: 5}
mix_1 = 2  #@param {type: "slider", min: -1, max: 5}
mix_2 = 2  #@param {type: "slider", min: -1, max: 5}
mix_3 = 2  #@param {type: "slider", min: -1, max: 5}
mix_4 = 2  #@param {type: "slider", min: -1, max: 5}
mix_5 = 2  #@param {type: "slider", min: -1, max: 5}
mix_6 = 2  #@param {type: "slider", min: -1, max: 5}
mix_7 = 2  #@param {type: "slider", min: -1, max: 5}
mix_8 = 2  #@param {type: "slider", min: -1, max: 5}
mix_9 = 2  #@param {type: "slider", min: -1, max: 5}
#@markdown ---
final_results = {'loop_sample': loop_sample, 'mix_0': mix_0, 'mix_1': mix_1, 'mix_2': mix_2, 
 'mix_3': mix_3, 'mix_4': mix_4, 'mix_5': mix_5, 'mix_6': mix_6, 'mix_7': mix_7,
 'mix_8': mix_8, 'mix_9': mix_9}
final_results.update({'hcom_{}'.format(i): list(dict_10.values())[i] for i in range(len(dict_10))})
final_results